In [1]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import optuna

In [1]:
 pip install optuna

     |████████████████████████████████| 409 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 226 kB 40.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 15.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
features = pd.read_csv('acsincome_ca_features.csv')
print(features)

        AGEP  COW  SCHL  MAR    OCCP   POBP  RELP  WKHP  SEX  RAC1P
0       30.0  6.0  14.0  1.0  9610.0    6.0  16.0  40.0  1.0    8.0
1       21.0  4.0  16.0  5.0  1970.0    6.0  17.0  20.0  1.0    1.0
2       65.0  2.0  22.0  5.0  2040.0    6.0  17.0   8.0  1.0    1.0
3       33.0  1.0  14.0  3.0  9610.0   36.0  16.0  40.0  1.0    1.0
4       18.0  2.0  19.0  5.0  1021.0    6.0  17.0  18.0  2.0    1.0
...      ...  ...   ...  ...     ...    ...   ...   ...  ...    ...
195660  38.0  1.0  22.0  1.0  1021.0  210.0   0.0  40.0  1.0    6.0
195661  39.0  1.0  22.0  1.0  1021.0  210.0   1.0  40.0  2.0    6.0
195662  61.0  1.0  19.0  1.0  5240.0   17.0   0.0  45.0  1.0    1.0
195663  69.0  7.0  24.0  1.0  2040.0  207.0   0.0  45.0  1.0    6.0
195664  40.0  1.0  17.0  1.0  9600.0  303.0   0.0  40.0  1.0    8.0

[195665 rows x 10 columns]


In [3]:
# Lire le fichier PHP ligne par ligne et stocker les valeurs dans une liste
with open('view.php', 'r') as file:
    lines = file.readlines()

# Supprimer les caractères de nouvelle ligne (\n) et convertir le texte en booléens
label = [line.strip() == 'True' for line in lines]

# Créer un DataFrame pandas pour les labels 
labels = pd.DataFrame(label, columns=['PINCP'])

# Afficher les 10 premières lignes du DataFrame
#print(df_labels.head(10))
print(labels)

        PINCP
0       False
1       False
2       False
3       False
4       False
...       ...
195661   True
195662   True
195663   True
195664  False
195665  False

[195666 rows x 1 columns]


In [4]:
# Mélanger les indices
indices = shuffle(range(len(features)), random_state=1)

# Utiliser les indices mélangés pour obtenir les nouvelles versions de X_all et y_all
X_all = features.iloc[indices]
y_all = labels.iloc[indices]

# only use the first N samples to limit training time
num_samples = int(len(X_all) * 0.1)
X, y = X_all.iloc[:num_samples], y_all.iloc[:num_samples]


In [11]:
print(X)
print(y)

        AGEP  COW  SCHL  MAR    OCCP   POBP  RELP  WKHP  SEX  RAC1P
58236   60.0  3.0  19.0  1.0  4220.0    1.0   1.0  48.0  2.0    1.0
101865  46.0  1.0  20.0  1.0  9645.0  156.0   1.0  40.0  2.0    1.0
96130   51.0  2.0  21.0  3.0  2300.0   29.0  12.0  30.0  2.0    1.0
43300   54.0  7.0  19.0  1.0  1108.0    6.0   1.0  50.0  1.0    1.0
102801  54.0  3.0  22.0  1.0  2310.0    6.0   0.0  79.0  2.0    1.0
...      ...  ...   ...  ...     ...    ...   ...   ...  ...    ...
66937   51.0  4.0  21.0  1.0   630.0    6.0   0.0  40.0  2.0    1.0
79289   32.0  5.0  22.0  1.0  9030.0   47.0   0.0  40.0  1.0    1.0
88966   67.0  1.0  19.0  1.0   220.0   48.0   0.0  40.0  1.0    1.0
33846   47.0  1.0  11.0  3.0  7700.0  303.0   2.0  46.0  1.0    8.0
100803  33.0  1.0  21.0  5.0  6050.0    6.0  13.0  60.0  1.0    9.0

[19566 rows x 10 columns]
        PINCP
58236    True
101865  False
96130   False
43300    True
102801   True
...       ...
66937   False
79289   False
88966   False
33846   False
100

In [5]:
# Divisez les données en un ensemble d'entraînement (80%) et un ensemble de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Affichez la taille des ensembles d'entraînement et de test
print("Taille de l'ensemble d'entraînement :", len(X_train))
print("Taille de l'ensemble de test :", len(X_test))


Taille de l'ensemble d'entraînement : 15652
Taille de l'ensemble de test : 3914


In [6]:
# Créer une liste de modèles
models = [
    ("SVM", SVC()),
    ("Adaboost", AdaBoostClassifier()),
    ("GradientBoosting", GradientBoostingClassifier()),
    ("RandomForest", RandomForestClassifier())
]

# Boucle sur les modèles
for name, model in models:
    # Effectuer une cross-validation
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f"{name}: Mean Accuracy: {scores.mean()}, Standard Deviation: {scores.std()}")


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

SVM: Mean Accuracy: 0.5913621284832801, Standard Deviation: 0.00010200585736544117


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Adaboost: Mean Accuracy: 0.6267570813558734, Standard Deviation: 0.004594728943760265


/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GradientBoosting: Mean Accuracy: 0.6379372716205971, Standard Deviation: 0.007578602228625776


/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y wa

RandomForest: Mean Accuracy: 0.6127014100977243, Standard Deviation: 0.004905986784325695


In [7]:
for name, model in models:
    model_instance = model

    model_instance.fit(X_train, y_train)

   
    y_pred = model_instance.predict(X_test)


    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"Evaluation results for {name}:")
    print(f"Accuracy: {accuracy}")
    print("Classification Report:\n", report)
    print("Confusion Matrix:\n", conf_matrix)
    print("\n" + "="*50 + "\n")  # Juste une séparation visuelle entre les résultats des différents modèles


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarnin

Evaluation results for SVM:
Accuracy: 0.5978538579458355
Classification Report:
               precision    recall  f1-score   support

       False       0.60      1.00      0.75      2340
        True       0.00      0.00      0.00      1574

    accuracy                           0.60      3914
   macro avg       0.30      0.50      0.37      3914
weighted avg       0.36      0.60      0.45      3914

Confusion Matrix:
 [[2340    0]
 [1574    0]]


Evaluation results for Adaboost:
Accuracy: 0.6359223300970874
Classification Report:
               precision    recall  f1-score   support

       False       0.65      0.85      0.74      2340
        True       0.59      0.32      0.41      1574

    accuracy                           0.64      3914
   macro avg       0.62      0.58      0.57      3914
weighted avg       0.62      0.64      0.61      3914

Confusion Matrix:
 [[1987  353]
 [1072  502]]




/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Evaluation results for GradientBoosting:
Accuracy: 0.641543178334185
Classification Report:
               precision    recall  f1-score   support

       False       0.65      0.89      0.75      2340
        True       0.62      0.28      0.38      1574

    accuracy                           0.64      3914
   macro avg       0.63      0.58      0.57      3914
weighted avg       0.64      0.64      0.60      3914

Confusion Matrix:
 [[2073  267]
 [1136  438]]




/var/folders/3x/g6pl7yns3rngqdy2ymmf1wrr0000gn/T/ipykernel_3083/708083295.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_instance.fit(X_train, y_train)


Evaluation results for RandomForest:
Accuracy: 0.6083290751149719
Classification Report:
               precision    recall  f1-score   support

       False       0.64      0.77      0.70      2340
        True       0.52      0.37      0.43      1574

    accuracy                           0.61      3914
   macro avg       0.58      0.57      0.57      3914
weighted avg       0.59      0.61      0.59      3914

Confusion Matrix:
 [[1806  534]
 [ 999  575]]




In [ ]:
param_grid = {
    'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}

grid_search = GridSearchCV(SVC(), param_grid, cv=5, verbose = 2)
grid_search.fit(X_train, y_train)

print(f"Best hyperparameters for 'SVC': {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")


Fitting 5 folds for each of 6 candidates, totalling 30 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END ...............................C=0.1, kernel=linear; total time= 9.0min


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END ...............................C=0.1, kernel=linear; total time=12.9min


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:


# Définir les hyperparamètres que vous souhaitez rechercher pour chaque modèle
param_grid = {
    'SVM': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    'Adaboost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]},
    'GradientBoosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]},
    'RandomForest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
}

# Boucle sur les modèles et effectuer la recherche des hyperparamètres
for name, model in models:
    if name in param_grid:
        grid_search = GridSearchCV(model, param_grid[name], cv=5)
        grid_search.fit(X_train, y_train)

        print(f"Best hyperparameters for {name}: {grid_search.best_params_}")
        print(f"Best cross-validation score: {grid_search.best_score_}")


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 